In [39]:
import networkx as nx

def load_edge_servers(file, G):
    with open(file, 'r') as f:
        for line in f:
            server, ram, cpu, storage = line.strip().split(',')
            G.add_node(server, ram=int(ram), cpu=int(cpu), storage=int(storage))

def load_connections(file, G):
    with open(file, 'r') as f:
        for line in f:
            server1, server2, latency, cost, bandwidth = line.strip().split(',')
            G.add_edge(server1, server2, latency=int(latency), cost=int(cost), bandwidth=float(bandwidth))

def load_iot_users(file, G):
    iot_users = []
    with open(file) as f:
        for line in f:
            user, app = line.strip().split(',')
            iot_users.append((user, app))
            nearest_server = find_nearest_server(user, G)
            G.add_node(user)
            G.add_edge(user, nearest_server, latency=5, bandwidth=1000) # Adjust these parameters as needed
    return iot_users


def find_nearest_server(user, G):
    nearest_server = None
    min_latency = float('inf')
    for server in G.nodes():
        if 'ram' in G.nodes[server]: # Check if it's a server node
            latency = nx.shortest_path_length(G, source=user, target=server, weight='latency')
            if latency < min_latency:
                min_latency = latency
                nearest_server = server
    return nearest_server

def load_network(servers_file, connections_file, iot_users_file):
    G = nx.Graph()
    load_edge_servers(servers_file, G)
    load_connections(connections_file, G)
    iot_users = load_iot_users(iot_users_file, G)
    return G, iot_users

# Example usage
network, iot_users = load_network("servers.txt", "connections.txt", "iot_users.txt")


NodeNotFound: Node User1 not found in graph

In [ ]:
# Print edge servers
print("Edge Servers:")
for node, data in network.nodes(data=True):
    print(f"Server: {node}, RAM: {data['ram']}GB, CPU: {data['cpu']} cores, Storage: {data['storage']}GB")

# Print connections
print("\nConnections:")
for server1, server2, data in network.edges(data=True):
    print(f"From: {server1}, To: {server2}, Latency: {data['latency']}ms, Cost: {data['cost']}, Bandwidth: {data['bandwidth']}GBps")

# Print IoT users
print("\nIoT Users:")
for user, app in iot_users:
    print(f"User: {user}, Requested App: {app}")

# Print apps
print("\nApps:")
for app, values in apps.items():
    bw, latency = values
    print(f"App: {app}, Bandwidth: {bw}GBps, Latency: {latency}ms")

Edge Servers:
Server: Server1, RAM: 16GB, CPU: 8 cores, Storage: 500GB
Server: Server2, RAM: 32GB, CPU: 16 cores, Storage: 1000GB
Server: Server3, RAM: 64GB, CPU: 32 cores, Storage: 2000GB
Server: Server4, RAM: 16GB, CPU: 8 cores, Storage: 1500GB
Server: Server5, RAM: 32GB, CPU: 16 cores, Storage: 800GB
Server: Server6, RAM: 64GB, CPU: 32 cores, Storage: 2500GB

Connections:
From: Server1, To: Server2, Latency: 10ms, Cost: 5, Bandwidth: 1.0GBps
From: Server2, To: Server3, Latency: 15ms, Cost: 10, Bandwidth: 10.0GBps
From: Server3, To: Server4, Latency: 5ms, Cost: 3, Bandwidth: 2.0GBps
From: Server4, To: Server5, Latency: 12ms, Cost: 7, Bandwidth: 5.0GBps
From: Server5, To: Server6, Latency: 8ms, Cost: 4, Bandwidth: 1.0GBps

IoT Users:
User: User1, Requested App: App1
User: User2, Requested App: App2
User: User3, Requested App: App3
User: User4, Requested App: App1
User: User5, Requested App: App2

Apps:
App: App1, Bandwidth: 1.0GBps, Latency: 1.0ms
App: App2, Bandwidth: 0.5GBps, Latenc

In [ ]:
## Define Parameters

# Example parameters - these can be adjusted as needed
VNF_CHAIN_LENGTH = 3  # Number of VNFs in a chain
MAX_LATENCY = 5      # Maximum allowed latency in ms
MAX_COST = 10        # Maximum allowed cost


In [ ]:
# Placement

def place_vnfs(user, app, network, iot_users, apps):
    app_bw, app_latency = apps[app]
    nearest_servers = sorted(network.nodes(), key=lambda s: network[user][s]['latency']) # Sort by proximity
    
    vnf_chain = []
    for server in nearest_servers:
        server_resources = network.nodes[server]
        # Check if server has enough resources
        if server_resources['ram'] > 0 and server_resources['cpu'] > 0 and server_resources['storage'] > 0:
            # Check if link meets bandwidth and latency requirements
            link_data = network[user][server]
            if link_data['bandwidth'] >= app_bw and link_data['latency'] <= app_latency:
                # Place VNF and update server resources
                vnf_chain.append(server)
                server_resources['ram'] -= 1  # Adjust based on VNF requirements
                server_resources['cpu'] -= 1
                server_resources['storage'] -= 1
                # Check if VNF chain is complete
                if len(vnf_chain) == VNF_CHAIN_LENGTH:
                    break
    
    return vnf_chain


In [ ]:
# Iterate through IoT users and place VNFs
for user, app in iot_users:
    vnf_chain = place_vnfs(user, app, network, iot_users, apps)
    print(f"User: {user}, App: {app}, VNF Chain: {vnf_chain}")


KeyError: 'User1'

In [ ]:
print(iot_users)
print(network.nodes())

[('User1', 'App1'), ('User2', 'App2'), ('User3', 'App3'), ('User4', 'App1'), ('User5', 'App2')]
['Server1', 'Server2', 'Server3', 'Server4', 'Server5', 'Server6']


In [ ]:
for edge in network.edges(data=True):
    print(edge)


('Server1', 'Server2', {'latency': 10, 'cost': 5, 'bandwidth': 1.0})
('Server2', 'Server3', {'latency': 15, 'cost': 10, 'bandwidth': 10.0})
('Server3', 'Server4', {'latency': 5, 'cost': 3, 'bandwidth': 2.0})
('Server4', 'Server5', {'latency': 12, 'cost': 7, 'bandwidth': 5.0})
('Server5', 'Server6', {'latency': 8, 'cost': 4, 'bandwidth': 1.0})
